# Este notebook muestra como utilizar el modelo **gpt2-small-spanish** para generación de lenguaje

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Carga de librerías y preparación del modelo

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, set_seed
from torch.cuda import set_device, current_device
#from fastai.text.all import *

Las librerías necesarias para poder ejecutar el ejemplo que te presentamos son _transformers_, y _pytorch_. Puedes ver en la salida de la siguiente celda las versiones que fueron utilizadas para correr este ejemplo 

In [3]:
import platform
from importlib_metadata import version
print("Python:", platform.python_version())
print("transformers:", version("transformers"))
print('pytorch:', version('torch'))

Python: 3.7.9
transformers: 4.2.0.dev0
pytorch: 1.7.1


Después de importar las ilbrerías, asignamos el GPU que querramos utilizar para correr el modelo.

In [4]:
gpu = 0
set_device(gpu)
print(f'cuda device: {current_device()}')

cuda device: 0


Para ejecutar este ejemplo, utilizamos una tarjeta GeForce GTX1080Ti (aparecen cuatro pero asignamos solo la primera, con índice 0)

In [5]:
!nvidia-smi

Mon Mar 15 00:59:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 19%   30C    P0    58W / 250W |      2MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 18%   

Ahora cargamos el tokenizer y el modelo que se encuentran alojados en la plataforma de huggingface. Para esto utilizamos la función de _from\_pretrained_, utilizando como único parametro el nombre del modelo en un string "datificate/gpt2-small-spanish"

In [6]:
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")
set_seed(1212) #para asegurar repetibilidad de los ejemplos

Cargamos el modelo a la GPU

In [7]:
model.to('cuda');

## Ejemplo de unicornios

Uno de los famosos ejemplos utilizados en la <a href="https://openai.com/blog/better-language-models/" >presentación del modelo GPT2</a> es la generación de texto de unicornios. Aquí utilizamos una versión traducida al español.

In [8]:
texto_unicornios = f'En un hallazgo impactante, el científico descubrió un rebaño de unicornios que vivían en un valle remoto, anteriormente inexplorado, ' \
                   f'en las montañas de los Andes. Aún más sorprendente para los investigadores fue el hecho de que los unicornios hablaban perfecto español.'

In [9]:
largo_maximo = 250
largo_minimo = 200

Transformamos el texto a tokens, que son representaciones numéricas del texto utilizando el vocabulario ene español con el que el modelo fue entrenado.

In [10]:
input_ids = tokenizer.encode(texto_unicornios, return_tensors='pt').to('cuda')

texto_reconstruido = tokenizer.decode(tokenizer.encode(texto_unicornios))

print('Identificadores de los tokens:\n', input_ids)
print('\nTexto reconstruido:\n', texto_reconstruido)

Identificadores de los tokens:
 tensor([[  507,   298, 18617, 10881,   840,    12,   284,  6548,  7223,   298,
         13719,   768,   258,   298, 39113,   496,   301, 10166,   278,   298,
          4654, 27646,    12,  5144, 11090, 40911,   326,    12,   278,   347,
          6674,   258,   312, 10323,    14, 22232,   436, 15973,   372,   312,
          7651,   405,   284,  1633,   258,   301,   312,   298, 39113,   496,
         24085, 14771,  1119,    14]], device='cuda:0')

Texto reconstruido:
 En un hallazgo impactante, el científico descubrió un rebaño de unicornios que vivían en un valle remoto, anteriormente inexplorado, en las montañas de los Andes. Aún más sorprendente para los investigadores fue el hecho de que los unicornios hablaban perfecto español.


In [11]:
%%time
#set top_k = 40 and num_return_sequences = 3
sample_outputs = model.generate(input_ids, pad_token_id=0, #50256,
                                   do_sample=True, 
                                   max_length=largo_maximo, 
                                   min_length=largo_minimo,
                                   top_k=40,
                                   num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
    print(">> Texto Generado {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')

>> Texto Generado 1

En un hallazgo impactante, el científico descubrió un rebaño de unicornios que vivían en un valle remoto, anteriormente inexplorado, en las montañas de los Andes. Aún más sorprendente para los investigadores fue el hecho de que los unicornios hablaban perfecto español.

A su vez, el descubrimiento de la especie "N. scelanoides" había llevado al editor de ciencia de la Universidad de Toronto, William King, a la conclusión de que "nadie está de acuerdo en quién es el "Unicornio del Pacífico Norte del mundo". En consecuencia, King, en su artículo de 1996 "La hipótesis de King y las dificultades de la hipótesis de King", sugirió que el nombre del género "Aeolophora" no debe ser confundido con las lenguas de los indígenas de Norteamérica.

En 1996, King propuso un nuevo género sinonimizado, "Aeolophora alba", que resultó ser en una revisión bastante amplia a partir de un estudio inicial del género "Aeolophora", que incluía a muchas especies diferentes. En el momento en 

## Ejemplo sobre el coronavirus

Para mostrar un ejemplo más, generamos varios textos hablando sober el coronavirus

In [15]:
texto_corona = f'Es importante tener en cuenta las medidas de higiene en tiempos de pandemia por coronavirus. '

In [16]:
input_ids_corona = tokenizer.encode(texto_corona, return_tensors='pt').to('cuda')


texto_reconstruido = tokenizer.decode(tokenizer.encode(texto_corona))

print('Identificadores de los tokens:\n', input_ids)
print('\nTexto reconstruido:\n', texto_reconstruido)

Identificadores de los tokens:
 tensor([[  507,   298, 18617, 10881,   840,    12,   284,  6548,  7223,   298,
         13719,   768,   258,   298, 39113,   496,   301, 10166,   278,   298,
          4654, 27646,    12,  5144, 11090, 40911,   326,    12,   278,   347,
          6674,   258,   312, 10323,    14, 22232,   436, 15973,   372,   312,
          7651,   405,   284,  1633,   258,   301,   312,   298, 39113,   496,
         24085, 14771,  1119,    14]], device='cuda:0')

Texto reconstruido:
 Es importante tener en cuenta las medidas de higiene en tiempos de pandemia por coronavirus. 


In [17]:
%%time

sample_outputs = model.generate(input_ids_corona, pad_token_id=0,
                                   do_sample=True, 
                                   max_length=largo_maximo, 
                                   min_length=largo_minimo,
                                   top_k=40,
                                   num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
    print(">> Texto Generado {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')

>> Texto Generado 1

Es importante tener en cuenta las medidas de higiene en tiempos de pandemia por coronavirus. 
El Gobierno estadounidense declaró en abril de 2020 que "los trabajadores de salud pública estadounidenses deben enfrentar y trabajar en condiciones seguras" en todo el mundo para que cualquier voluntario que se enfermen se le presione a la salud para mantener la salud.

El 12 de junio de 2020, la pandemia de coronavirus en la ONU se incrementó al 31 de marzo, y el gobierno de Estados Unidos anunció que había alcanzado los 12 millones de personas en menos de seis meses. La ciudad de Wuhan, ubicada con la capital, es el centro del país. 

El 8 de julio del 2020, la Agencia Europea de Servicios de Emergencia fue establecida, con su responsabilidad, controlar el aislamiento de Wuhan del estado de emergencia. El 14 de julio se había establecido el toque de queda en todo el territorio continental.

Las escuelas, las cárceles de salud y otros centros de salud fueron cerrados y e